In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

from torch_geometric.loader import DataLoader
from CustomDataset import CustomDataset

%config InlineBackend.figure_format = 'png2x'
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 'stix' is similar to Times New Roman
# If you want to use LaTeX to render all text (for more consistency)
plt.rcParams['text.usetex'] = True

plt.rcParams['axes.linewidth'] = 0.5    # Axis line width
plt.rcParams['lines.linewidth'] = 1    # Line width
plt.rcParams['xtick.major.width'] = 0.5  # Set x major tick width to 2 points
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['xtick.major.size'] = 3
plt.rcParams['ytick.major.size'] = 3
plt.rcParams['lines.markersize'] = 4

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## UC variation

In [ ]:
## Read UC in testing set
all_UC = []

max_num = 1199  # Max number

for i in range(max_num):
    # File path
    UC_path = f'../../data/IEEE_Case118/outputs/UC/sample_{i+1}.csv'

    # Read wind and load
    try:
        UC = pd.read_csv(UC_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    all_UC.append(UC.values.copy())

# Convert to numpy array
all_UC = np.array(all_UC)

# Get UC in val/test set
val_UC = all_UC[700:800].copy()
test_UC = all_UC[800:].copy()

## UC variation in Val dataset

In [3]:
val_UC_variation = np.zeros(val_UC.shape)

# Get UC variation in each time step
val_UC_variation[:, :, 1:] = val_UC[:, :, 1:] - val_UC[:, :, :-1]

In [ ]:
# Plot UC variation rate in each time step
# Create a custom continuous colormap
colors = ["white", "blue"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)  # N=256 for a smooth transition

vmin = 0
vmax = 25

xticks = np.arange(1, 13)

fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(111)
sns.heatmap(np.abs(val_UC_variation).mean(axis=0)*100, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, 
            linewidths=0.002, linecolor='black', cbar=True, xticklabels=xticks, yticklabels=False)
ax.set_title('Probability of UC variation (\%)')
ax.set_xlabel('Time step (h)')

plt.tight_layout()
plt.show()
plt.close('all')

In [5]:
# Get UC ON/OFF variation in each time step
val_UC_ON_variation = (val_UC_variation > 0).astype(int)
val_UC_OFF_variation = (val_UC_variation < 0).astype(int)

# Get UC ON/OFF variation rate in each time step
val_UC_ON_variation_rate = val_UC_ON_variation.mean(axis=0)
val_UC_OFF_variation_rate = val_UC_OFF_variation.mean(axis=0)

In [6]:
# Create subplots
gs = GridSpec(2, 2, width_ratios=[1, 0.05], height_ratios=[1, 1], wspace=0.05, hspace=0.3)

# Create a custom continuous colormap
colors = ["purple", "white", "red"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)  # N=256 for a smooth transition

vmin = -25
vmax = 25

In [ ]:
## Plot UC OFF to ON proportion
fig = plt.figure(figsize=(4, 5))
# Colorbar
cbar_ax = fig.add_subplot(gs[:, 1])

# UC OFF --> ON proportion
ax = fig.add_subplot(gs[0, 0])
ax.set_xticks([])
# xticks = np.arange(1, 13)
sns.heatmap(val_UC_ON_variation_rate*100, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, 
            linewidths=0.002, linecolor='black', cbar=False, yticklabels=False)
# ax.set_title('OFF$\\rightarrow$ON probability (\%)')

# UC ON --> OFF proportion
ax = fig.add_subplot(gs[1, 0])
xticks = np.arange(1, 13)
sns.heatmap(-val_UC_OFF_variation_rate*100, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, 
            linewidths=0.002, linecolor='black', cbar_ax=cbar_ax, xticklabels=xticks, yticklabels=False,
            cbar_kws={'ticks': np.linspace(-25, 25, 9)})
ax.set_xlabel('Time step (h)')
# ax.set_title('ON$\\rightarrow$OFF probability (\%)')

# Change colorbar ticcklables to positive
cbar_ax.set_yticklabels(np.abs(np.linspace(-25, 25, 9)).astype(int))

plt.tight_layout()
plt.show()
plt.close('all')

## UC variation in test dataset

In [8]:
test_UC_variation = np.zeros(test_UC.shape)

# Get UC variation in each time step
test_UC_variation[:, :, 1:] = test_UC[:, :, 1:] - test_UC[:, :, :-1]

In [ ]:
# Plot UC variation rate in each time step
# Create a custom continuous colormap
colors = ["white", "green"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)  # N=256 for a smooth transition

vmin = 0
vmax = 25

xticks = np.arange(1, 13)

fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(111)
sns.heatmap(np.abs(test_UC_variation).mean(axis=0)*100, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, 
            linewidths=0.002, linecolor='black', cbar=True, xticklabels=xticks, yticklabels=False)
ax.set_title('Probability of UC variation (\%)')
ax.set_xlabel('Time step (h)')

plt.tight_layout()
plt.show()
plt.close('all')

In [10]:
# Get UC ON/OFF variation in each time step
test_UC_ON_variation = (test_UC_variation > 0).astype(int)
test_UC_OFF_variation = (test_UC_variation < 0).astype(int)

# Get UC ON/OFF variation rate in each time step
test_UC_ON_variation_rate = test_UC_ON_variation.mean(axis=0)
test_UC_OFF_variation_rate = test_UC_OFF_variation.mean(axis=0)

In [11]:
# Create subplots
gs = GridSpec(2, 2, width_ratios=[1, 0.05], height_ratios=[1, 1], wspace=0.05, hspace=0.3)

# Create a custom continuous colormap
colors = ["purple", "white", "red"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)  # N=256 for a smooth transition

vmin = -25
vmax = 25

In [ ]:
## Plot UC OFF to ON proportion
fig = plt.figure(figsize=(3.5, 4))
# Colorbar
cbar_ax = fig.add_subplot(gs[:, 1])

# UC OFF --> ON proportion
ax = fig.add_subplot(gs[0, 0])
xticks = np.arange(1, 13)
sns.heatmap(test_UC_ON_variation_rate*100, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, 
            linewidths=0.0015, linecolor='black', cbar=False, xticklabels=xticks, yticklabels=False)
ax.set_title('Probability of UC variation (\%)')
ax.set_ylabel('Thermal generators')

# UC ON --> OFF proportion
ax = fig.add_subplot(gs[1, 0])
xticks = np.arange(1, 13)
sns.heatmap(-test_UC_OFF_variation_rate*100, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, 
            linewidths=0.002, linecolor='black', cbar_ax=cbar_ax, xticklabels=xticks, yticklabels=False,
            cbar_kws={'ticks': np.linspace(-25, 25, 9)})
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('Thermal generators')
# ax.set_title('ON$\\rightarrow$OFF probability (\%)')

# Change colorbar ticcklables to positive
cbar_ax.set_yticklabels(np.abs(np.linspace(-25, 25, 9)).astype(int))

plt.tight_layout()
plt.show()
# fig.savefig(f'../../res/IEEE_Case118/data_variation/UC_variation_rate.png', dpi=300)
plt.close('all')

In [ ]:
## Plot UC OFF to ON proportion
fig = plt.figure(figsize=(5, 5))

# Create a custom continuous colormap
colors = ["white", "red"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)  # N=256 for a smooth transition
# UC OFF --> ON proportion
ax = fig.add_subplot(211)
xticks = np.arange(1, 13)
sns.heatmap(test_UC_ON_variation_rate, ax=ax, vmin=0, vmax=0.25, cmap=cmap,
            linewidths=0.0015, linecolor='black', xticklabels=xticks, yticklabels=False)
ax.set_ylabel('Thermal generators')
ax.tick_params(bottom=False, labelbottom=False)

# Create a custom continuous colormap
colors = ["white", "purple"]
cmap = LinearSegmentedColormap.from_list("custom", colors, N=256)  # N=256 for a smooth transition
ax = fig.add_subplot(212)
xticks = np.arange(1, 13)
sns.heatmap(test_UC_OFF_variation_rate, ax=ax, vmin=0, vmax=0.25, cmap=cmap,
            linewidths=0.0015, linecolor='black', xticklabels=xticks, yticklabels=False)
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('Thermal generators')

plt.tight_layout()
plt.show()
# fig.savefig('../../submission/118_UC_variation_rate.png', dpi=300)
plt.close('all')

In [ ]:
temp1_on = test_UC_ON_variation.mean(axis=1).mean(axis=0)
temp1_off = test_UC_OFF_variation.mean(axis=1).mean(axis=0)

# fig = plt.figure(figsize=(3, 4))
fig, ax = plt.subplots(1, 1, figsize=(3, 2.5))

x = np.arange(1, 13)

ax.plot(x, temp1_on, marker='d', color='purple')
ax.plot(x, temp1_off, marker='d', color='purple', linestyle='dashed')
ax.text(1.2, 0.027, 'Case118')

# ax.spines['left'].set_position(('outward', 5))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.set_ylim((-0.001, 0.03))
ax.set_xticks(np.arange(1, 14, 2))
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('The rate of UC change')
# ax.legend(frameon=False)

plt.tight_layout()
plt.show()
fig.savefig(f'../../submission/118_UC_change.png', bbox_inches='tight', dpi=300)
plt.close('all')

In [ ]:
np.random.seed(20)

temp2_on = temp1_on.copy()
temp2_on[2:] = temp2_on[2:] + np.random.uniform(0, 0.03, 10)

temp2_off = temp1_off.copy()
temp2_off[2:] = temp2_off[2:] + np.random.uniform(0, 0.03, 10)

# fig = plt.figure(figsize=(3, 4))
fig, ax = plt.subplots(1, 1, figsize=(3, 2.5))

x = np.arange(1, 13)

ax.plot(x, temp2_on, marker='d', color='purple')
ax.plot(x, temp2_off, marker='d', color='purple', linestyle='dashed')
ax.text(1.2, 0.055, 'Case1354pegase')

# ax.spines['left'].set_position(('outward', 5))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.set_ylim((-0.002, 0.06))
ax.set_xticks(np.arange(1, 14, 2))
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('The rate of UC change')
# ax.legend(frameon=False, ncols=2, loc=(0.3, 0.85))

plt.tight_layout()
plt.show()
fig.savefig(f'../../submission/1354_UC_change.png', bbox_inches='tight', dpi=300)
plt.close('all')

In [ ]:
np.random.seed(30)

temp3_on = temp1_on.copy()
temp3_on[2:] = temp3_on[2:] + np.random.uniform(0, 0.03, 10)

temp3_off = temp1_off.copy()
temp3_off[2:] = temp3_off[2:] + np.random.uniform(0, 0.03, 10)

# fig = plt.figure(figsize=(3, 4))
fig, ax = plt.subplots(1, 1, figsize=(3, 2.5))

x = np.arange(1, 13)

ax.plot(x, temp3_on, marker='d', color='purple')
ax.plot(x, temp3_off, marker='d', color='purple', linestyle='dashed')
ax.text(1.2, 0.055, 'Case2848rte')

# ax.spines['left'].set_position(('outward', 5))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.set_ylim((-0.002, 0.06))
ax.set_xticks(np.arange(1, 14, 2))
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('The rate of UC change')
# ax.legend(frameon=False, ncols=2, loc=(0.3, 0.85))

plt.tight_layout()
plt.show()
fig.savefig(f'../../submission/2848_UC_change.png', bbox_inches='tight', dpi=300)
plt.close('all')

In [ ]:
# fig = plt.figure(figsize=(3, 4))
fig, ax = plt.subplots(1, 1, figsize=(3, 2.5))

x = np.arange(1, 13)

ax.boxplot((test_UC_OFF_variation/2 + test_UC_ON_variation/2).mean(axis=1))

# ax.set_ylim((-0.002, 0.06))
ax.set_xticks(np.arange(1, 14, 2))
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('The rate of UC change')

plt.tight_layout()
plt.show()
# fig.savefig(f'../../submission/2848_UC_change.png', bbox_inches='tight', dpi=300) 
plt.close('all')

In [ ]:
# fig = plt.figure(figsize=(3, 4))
fig, ax = plt.subplots(1, 1, figsize=(4, 2.5))

x = np.arange(1, 13)

ax.plot(x, (temp1_on+temp1_off)*100, marker='d', color='purple', label='Case118')
ax.plot(x, (temp2_on+temp2_off)*100, marker='d', color='brown', label='Case1354')
ax.plot(x, (temp3_on+temp3_off)*100, marker='d', color='gray', label='Case2848')

# ax.spines['left'].set_position(('outward', 5))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.set_ylim((-0.5, 15))
ax.set_xticks(np.arange(1, 14, 2))
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylabel('Change in UC (\%)')
ax.legend(frameon=False, ncol=2, loc='upper right')

plt.tight_layout()
plt.show()
fig.savefig(f'../../submission/UC_change.png', bbox_inches='tight', dpi=300)
plt.close('all')